# Program to fit a straight line to the set of arbitrary strips

In [1]:
import numpy as np
from scipy.optimize import minimize
from numpy.random import default_rng
rng = default_rng()

In [2]:
class RANSAC:
    def __init__(self, n=5, k=100, t=0.005, d=8, model=None, loss=None, metric=None):
        self.n = n              # `n`: Minimum number of data points to estimate parameters
        self.k = k              # `k`: Maximum iterations allowed
        self.t = t              # `t`: Threshold value to determine if points are fit well
        self.d = d              # `d`: Number of close data points required to assert model fits well
        self.loss = loss        # `loss`: function of `y_true` and `y_pred` that returns a vector
        self.metric = metric    # `metric`: function of `y_true` and `y_pred` and returns a float
        self.fun = 0.0          # chi_2 of the fit
        self.x = np.zeros(4)        # fit result
        self.jac = np.zeros(4)      # resulting jacobian
        self.bestStrips = np.empty  # strips used for fitting
        self.currStrips = np.empty  
        
        
    # calculate total Chi2
    def chiSquare(self, par):
        # par = [Ax, Ay, Bx, By]
        # track:  X = Ax*z +Bx, Y = Ay*z +By
        #  
        
        Z0 = 0
        Z1 = 100
        tr0 = (par[0]*Z0+par[2], par[1]*Z0+par[3], Z0)
        tr1 = (par[0]*Z1+par[2], par[1]*Z1+par[3], Z1)
           
        chi2 = 0
        for iStrip in self.currStrips:
            chi2 = chi2 + self.distance_between_lines(tr0, tr1, iStrip[0], iStrip[1])**2
        return chi2           
        
    # calculate single strip Chi2
    def chiSquareSingle(self, par, kStrip):
        # par = [Ax, Ay, Bx, By]
        # track:  X = Ax*z +Bx, Y = Ay*z +By
        #  
        
        Z0 = 0
        Z1 = 100
        tr0 = (par[0]*Z0+par[2], par[1]*Z0+par[3], Z0)
        tr1 = (par[0]*Z1+par[2], par[1]*Z1+par[3], Z1)
           
        chi2 = self.distance_between_lines(tr0, tr1, kStrip[0], kStrip[1])**2
        return chi2     
    
    # calculate distance between two lines in 3D
    def distance_between_lines(self, a, b, c, d):
        # a - first point of the first line in 3D (x,y,z)
        # b - second point of the first line
        # c - first point of the second line
        # d - second point of the second line
        # https://math.stackexchange.com/questions/210848/finding-the-shortest-distance-between-two-lines
        v1 = np.array(b) - np.array(a)
        v2 = np.array(d) - np.array(c)
        cross = np.cross(v1, v2)
        if cross.any() != 0:
            # lines not parallel
            cross = cross/np.linalg.norm(cross)
            return np.dot(cross,np.subtract(c,a))
        else:
            # lines parallel - calculate a distance from point c to line (a,b)
            # https://stackoverflow.com/questions/39840030/distance-between-point-and-a-line-from-two-points
            return np.linalg.norm(np.cross(np.subtract(b,a),np.subtract(c,a)))/np.linalg.norm(np.subtract(b,a))
        
    #actual fit    
    def fit(self, strips):
        
        par = np.zeros(4)
        # fit to all points
        self.currStrips = strips
        resBest = minimize(self.chiSquare, par, method='SLSQP')

        
        for _ in range(self.k):
            ids = rng.permutation(strips.shape[0])

            self.currStrips = strips[ids[: self.n]]
            res = minimize(self.chiSquare, par, method='SLSQP')

            # check whether currStrips are below threshold
            maxChi2 = 0;
            for kStrip in self.currStrips:
                chi2 = self.chiSquareSingle(res.x, kStrip)
                if chi2 > maxChi2:
                    maxChi2 = chi2
                    if chi2 > self.t:
                        break
            
            if maxChi2 < self.t:
                # add points that are close to the fitted model
                for kStrip in strips[ids[self.n:]]:
                    if self.chiSquareSingle(res.x, kStrip) < self.t: 
                        self.currStrips = np.append(self.currStrips, kStrip).reshape(self.currStrips.shape[0]+1,
                            self.currStrips.shape[1],self.currStrips.shape[2])
                        
            if self.currStrips.shape[0] >= self.d:
                res = minimize(self.chiSquare, par, method='SLSQP')
                if res.fun < resBest.fun:
                    resBest = res
                    self.bestStrips = self.currStrips
                
                
                       
        self.x = resBest.x
        self.jac = resBest.jac
        self.fun = resBest.fun
        print(resBest)
              
        return self

# Example of  usage

In [3]:
# Arbitrary 10 layer strip telescope with randomly rotated strips
# Hit generator

import numpy as np
import random as rnd

In [4]:
# error
sigma = 0.01
# n noisy hits
n_noise = 2

n = 10 # n-layers
Z = np.arange(0, n)

rnd.seed(10)
#track parameters
Ax = rnd.uniform(-0.1, 0.1)
Ay = rnd.uniform(-0.1, 0.1)
Bx = rnd.uniform(-1, 1)
By = rnd.uniform(-1, 1)

print(" Generated track parameters Ax, Ay, Bx, By = ", Ax, Ay, Bx, By)
#strip rotation angles
rot = np.random.uniform(0, np.pi, n)

print("Z and rotation angles of strips: ",Z, rot)

 Generated track parameters Ax, Ay, Bx, By =  0.014280518937982697 -0.014222189064977075 0.15618260226894076 -0.5878035357209965
Z and rotation angles of strips:  [0 1 2 3 4 5 6 7 8 9] [1.45494288 1.59672535 1.98253132 2.28328134 0.95663819 3.12947596
 0.99373546 0.84544686 1.62292514 0.98524946]


In [5]:
n = 10 # n-layers
# error
sigma = 0.001
# n noisy hits
n_noise = 10
# sigma of noisy hits
sigma_noise = 2
# half-length of a strip
R = 5.


In [6]:

hits = np.zeros([n+n_noise,3])
strips = np.zeros([n+n_noise,2,3])
rotx = np.zeros([n+n_noise])

# real hits
for i in range(n):
    hits[i,0] = Ax*Z[i]+Bx + rnd.gauss(0, sigma)
    hits[i,1] = Ay*Z[i]+By + rnd.gauss(0, sigma)
    hits[i,2] = Z[i]
    rotx[i] = rot[i]
    
# Adding noisy hits
for i in range(n,n+n_noise):
    hits[i,0] = rnd.gauss(0, sigma_noise)
    hits[i,1] = rnd.gauss(0, sigma_noise)
    hits[i,2] = Z[rnd.randint(0,n-1)]
    
# translate hits into strips fired
for i in range(n+n_noise):    
    strips[i,0,0] = hits[i,0]-R*np.cos(rotx[i])
    strips[i,0,1] = hits[i,1]-R*np.sin(rotx[i])
    strips[i,0,2] = hits[i,2]
    
    strips[i,1,0] = hits[i,0]+R*np.cos(rotx[i])
    strips[i,1,1] = hits[i,1]+R*np.sin(rotx[i])
    strips[i,1,2] = hits[i,2]
    
#print(hits)    
#print(strips) 

### Actual RANSAC fit

In [7]:
regressor = RANSAC()
regressor.fit(strips)


print("Fitted chi_2: ",regressor.fun, "No. of hits: ", regressor.bestStrips.shape[0])
print("Fit results Ax, Ay, Bx, By: ",regressor.x," +- ", regressor.jac)

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 6.085023810370193e-06
       x: [ 1.407e-02 -1.469e-02  1.567e-01 -5.862e-01]
     nit: 8
     jac: [-5.739e-03 -5.931e-03 -5.255e-04 -9.313e-04]
    nfev: 48
    njev: 8
Fitted chi_2:  6.085023810370193e-06 No. of hits:  10
Fit results Ax, Ay, Bx, By:  [ 0.01406628 -0.01468748  0.1566922  -0.5862025 ]  +-  [-0.00573879 -0.00593057 -0.00052548 -0.00093131]
